In [1]:
import os
import json
import time
from pathlib import Path
from typing import List, Dict, Any
from mlx_lm import load, generate
from dotenv import load_dotenv

/opt/homebrew/Caskroom/miniforge/base/envs/unrc-cs-thesis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
ROOT_PATH = os.environ.get("ROOT_PATH")
if not ROOT_PATH:
    raise ValueError("ROOT_PATH environment variable not set. Please set it in your .env file.")

PRED_PATH = f"{ROOT_PATH}/data/predictions"

In [17]:
def load_model(model_path: str = "mlx-community/Llama-3.2-3B-Instruct-4bit", adapter_path: str = None):
    """Load the MLX model and tokenizer, optionally with LoRA adapter"""
    print(f"Loading model: {model_path}")
    
    if adapter_path:
        print(f"Loading with adapter: {adapter_path}")
        model, tokenizer = load(model_path, adapter_path=adapter_path)
        print("Model and adapter loaded successfully!")
    else:
        model, tokenizer = load(model_path)
        print("Model loaded successfully!")
    
    return model, tokenizer

def process_batch(
    prompts: List[str], 
    model, 
    tokenizer,
    max_tokens: int = 512
) -> List[Dict[str, Any]]:
    """Process a batch of prompts and return results"""
    results = []
    for i, prompt in enumerate(prompts):
        print(f"Processing prompt {i+1}/{len(prompts)}")
        
        start_time = time.time()
        
        try:
            # Generate response
            messages = [{"role": "user", "content": prompt}]
            message = tokenizer.apply_chat_template(
                messages, add_generation_prompt=True
            )
            response = generate(model, tokenizer, prompt=message, max_tokens=max_tokens, verbose=False)
            
            end_time = time.time()
            
            result = {
                "prompt_index": i,
                "prompt": prompt,
                "response": response,
                "generation_time": end_time - start_time,
                "status": "success"
            }
            
        except Exception as e:
            result = {
                "prompt_index": i,
                "prompt": prompt,
                "response": None,
                "generation_time": 0,
                "status": "error",
                "error": str(e)
            }
            print(f"Error processing prompt {i+1}: {e}")
        
        results.append(result)
        print(f"Completed in {result['generation_time']:.2f}s")
        print("-" * 50)
    
    return results

def save_results(results: List[Dict[str, Any]], output_file: str = "pred.sql"):
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_file, "w", encoding="utf-8") as f:
        for query in results:
            f.write(query['response'] + "\n")
    print(f"Results saved to {output_file}")

def load_prompts_from_file(file_path: str) -> List[str]:
    """Load prompts from a text file, JSON file, or JSONL file"""
    prompts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data = json.loads(line)
                if 'prompt' in data:
                    prompts.append(data['prompt'])
                else:
                    print(f"Warning: Line {line_num} missing 'prompt' field, skipping")
            except json.JSONDecodeError as e:
                print(f"Warning: Invalid JSON on line {line_num}, skipping: {e}")
    return prompts

In [22]:
prompts = ["Answer the question by SQLite SQL query only and with no explanation. You must minimize SQL execution time while ensuring correctness.\nSqlite SQL tables, with their properties:\nBreeds(breed_code, breed_name)\nCharges(charge_id, charge_type, charge_amount)\nSizes(size_code, size_description)\nTreatment_Types(treatment_type_code, treatment_type_description)\nOwners(owner_id, first_name, last_name, street, city, state, zip_code, email_address, home_phone, cell_number)\nDogs(dog_id, owner_id, abandoned_yn, breed_code, size_code, name, age, date_of_birth, gender, weight, date_arrived, date_adopted, date_departed)\nProfessionals(professional_id, role_code, first_name, street, city, state, zip_code, last_name, email_address, home_phone, cell_number)\nTreatments(treatment_id, dog_id, professional_id, treatment_type_code, date_of_treatment, cost_of_treatment)\nHere is some data information about database references.\nBreeds(breed_code[ESK, HUS, BUL], breed_name[Eskimo, Husky, Bulldog])\nCharges(charge_id[1, 2, 3], charge_type[Daily Accommodation, Drugs, Health Check], charge_amount[98, 322, 640])\nSizes(size_code[SML, MED, LGE], size_description[Small, Medium, Large])\nTreatment_Types(treatment_type_code[EXAM, VAC, WALK], treatment_type_description[Physical examination, Vaccination, Take for a Walk])\nOwners(owner_id[1, 2, 3], first_name[Nora, Melisa, Jaclyn], last_name[Haley, DuBuque, Stoltenberg], street[0647 Hintz Village Apt. 024, 1204 Mae Highway Apt. 107, 2635 Caitlyn Plains], city[Lake Tia, Port Reannamouth, Barrettburgh], state[Wisconsin, Virginia, Vermont], zip_code[93165, 45244, 02647], email_address[lynn81@example.org, ykris@example.com, hudson.favian@example.com], home_phone[1-682-845-0116x63235, (799)563-0260x454, 916.976.9480x13851], cell_number[478.978.0729, (722)768-5439x484, 481-483-9083x37986])\nDogs(dog_id[1, 2, 3], owner_id[3, 11, 1], abandoned_yn[1, 0, 0], breed_code[ESK, BUL, BUL], size_code[LGE, LGE, MED], name[Kacey, Hipolito, Mavis], age[6, 9, 8], date_of_birth[2012-01-27 05:11:53, 2013-02-13 05:15:21, 2008-05-19 15:54:49], gender[1, 0, 1], weight[7.57, 1.72, 8.04], date_arrived[2017-09-08 20:10:13, 2017-12-22 05:02:02, 2017-06-25 10:14:05], date_adopted[2018-03-06 16:32:11, 2018-03-25 08:12:51, 2018-03-07 21:45:43], date_departed[2018-03-25 06:58:44, 2018-03-25 02:11:32, 2018-03-25 10:25:46])\nProfessionals(professional_id[1, 2, 3], role_code[Employee, Employee, Employee], first_name[Taryn, Jayson, Olaf], street[6915 Oberbrunner Point Suite 491\nGleasonville, LA , 88665 Terence Lodge Apt. 904\nCorneliusfort, NC 194, 68589 Bradly Manor\nNew Audrey, IN 91497], city[West Heidi, North Odellfurt, Jaceton], state[Indiana, Connecticut, Wisconsin], zip_code[06646, 43129, 77129], last_name[Braun, Ullrich, Watsica], email_address[deanna.schuster@example.com, lucile.shanahan@example.org, uboehm@example.org], home_phone[+71(6)2898266914, +02(1)0259033559, 325-155-0801x7005], cell_number[(275)939-2435x80863, 889-940-2676, (369)908-7311x065])\nTreatments(treatment_id[1, 2, 3], dog_id[14, 4, 14], professional_id[9, 10, 4], treatment_type_code[WALK, VAC, EXAM], date_of_treatment[2018-03-19 04:39:54, 2018-03-15 20:25:34, 2018-03-08 05:26:23], cost_of_treatment[567, 147, 429])\nForeign key information of SQLite tables, used for table joins:\nDogs(owner_id) REFERENCES Owners(owner_id)\nDogs(owner_id) REFERENCES Owners(owner_id)\nDogs(size_code) REFERENCES Sizes(size_code)\nDogs(breed_code) REFERENCES Breeds(breed_code)\nTreatments(dog_id) REFERENCES Dogs(dog_id)\nTreatments(professional_id) REFERENCES Professionals(professional_id)\nTreatments(treatment_type_code) REFERENCES Treatment_Types(treatment_type_code)\nSome example pairs of questions and corresponding SQL queries are provided based on similar questions:\nHow many farms are there?\nSELECT count(*) FROM farm\nWhat is the average, minimum, and maximum age for all French singers?\nSELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France'\nShow the ID of the high schooler named Kyle.\nSELECT ID FROM Highschooler WHERE name = 'Kyle'\nhow many dogs have an age below the average?"]
# prompt = """### Answer the question by SQLite SQL query only and with no explanation. You must minimize SQL execution time while ensuring correctness.\n### Sqlite SQL tables, with their properties:\n# Products(product_id, parent_product_id, product_name, product_price, product_color, product_size, product_description)\n# Customers(customer_id, gender_code, customer_first_name, customer_middle_initial, customer_last_name, email_address, login_name, login_password, phone_number, address_line_1, town_city, county, country)\n# Customer_Payment_Methods(customer_id, payment_method_code)\n# Invoices(invoice_number, invoice_status_code, invoice_date)\n# Orders(order_id, customer_id, order_status_code, date_order_placed)\n# Order_Items(order_item_id, product_id, order_id, order_item_status_code)\n# Shipments(shipment_id, order_id, invoice_number, shipment_tracking_number, shipment_date)\n# Shipment_Items(shipment_id, order_item_id)\n#\n### Here is some data information about database references.\n#\n# Products(product_id[1, 2, 3], parent_product_id[8, 3, 1], product_name[Dell monitor, Dell keyboard, iPhone6s], product_price[795.62, 104, 560.93], product_color[Red, Yellow, Red], product_size[Medium, Medium, Small], product_description[Latest model!, Keyboard for games!, Second hand!])\n# Customers(customer_id[1, 2, 3], gender_code[Female, Male, Male], customer_first_name[Carmen, Jace, Vickie], customer_middle_initial[K, P, B], customer_last_name[Treutel, Mraz, Bergnaum], email_address[pgulgowski@example.com, zwisozk@example.org, herzog.imogene@example.org], login_name[murphy07, desmond.steuber, kihn.alfonso], login_password[58952d0e0d28de32db3b, 7ba2e47aa0904d9fbdbf, 83a1afbe21f5ca4cd2d5], phone_number[(253)336-6277, 628-468-4228x5917, 633-223-0975], address_line_1[646 Herzog Key Suite 702, 67899 Cassin Hollow Suite 071, 395 Christophe Trail], town_city[Port Madonnahaven, Port Korychester, Lornaland], county[Israel, Palau, Moldova], country[USA, USA, USA])\n# Customer_Payment_Methods(customer_id[15, 1, 10], payment_method_code[Direct Debit, Direct Debit, Direct Debit])\n# Invoices(invoice_number[1, 2, 3], invoice_status_code[Paid, Issued, Paid], invoice_date[2018-03-09 07:16:07, 2018-01-28 20:08:22, 2018-02-13 02:16:55])\n# Orders(order_id[1, 2, 3], customer_id[5, 13, 13], order_status_code[Cancelled, Part Completed, Cancelled], date_order_placed[2017-09-17 16:13:07, 2017-10-14 12:05:48, 2017-09-10 08:27:04])\n# Order_Items(order_item_id[1, 2, 3], product_id[4, 3, 2], order_id[8, 4, 7], order_item_status_code[Delivered, Out of Stock, Delivered])\n# Shipments(shipment_id[1, 2, 3], order_id[7, 6, 9], invoice_number[5, 2, 4], shipment_tracking_number[6900, 3499, 5617], shipment_date[2018-02-28 00:04:11, 2018-03-07 01:57:14, 2018-03-18 22:23:19])\n# Shipment_Items(shipment_id[4, 7, 15], order_item_id[4, 14, 9])\n#\n### Foreign key information of SQLite tables, used for table joins:\n#\n# Customer_Payment_Methods(customer_id) REFERENCES Customers(customer_id)\n# Orders(customer_id) REFERENCES Customers(customer_id)\n# Order_Items(order_id) REFERENCES Orders(order_id)\n# Order_Items(product_id) REFERENCES Products(product_id)\n# Shipments(order_id) REFERENCES Orders(order_id)\n# Shipments(invoice_number) REFERENCES Invoices(invoice_number)\n# Shipment_Items(order_item_id) REFERENCES Order_Items(order_item_id)\n# Shipment_Items(shipment_id) REFERENCES Shipments(shipment_id)\n#\n### Some example pairs of questions and corresponding SQL queries are provided based on similar questions:\n### How many farms are there?\nSELECT count(*) FROM farm\n### What is the average, minimum, and maximum age for all French singers?\nSELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'\n### Show the ID of the high schooler named Kyle.\nSELECT ID FROM Highschooler WHERE name  =  'Kyle'\n#\n### What are the names and colors of all products that have been shipped?"""
# prompt = """
# ### Answer the question by SQLite SQL query only and with no explanation. You must minimize SQL execution time while ensuring correctness.
# ### Sqlite SQL tables, with their properties:
# # Products(product_id, parent_product_id, product_name, product_price, product_color, product_size, product_description)
# # Customers(customer_id, gender_code, customer_first_name, customer_middle_initial, customer_last_name, email_address, login_name, login_password, phone_number, address_line_1, town_city, county, country)
# # Customer_Payment_Methods(customer_id, payment_method_code)
# # Invoices(invoice_number, invoice_status_code, invoice_date)
# # Orders(order_id, customer_id, order_status_code, date_order_placed)
# # Order_Items(order_item_id, product_id, order_id, order_item_status_code)
# # Shipments(shipment_id, order_id, invoice_number, shipment_tracking_number, shipment_date)
# # Shipment_Items(shipment_id, order_item_id)
# #
# ### Here is some data information about database references.
# #
# # Products(product_id[1, 2, 3], parent_product_id[8, 3, 1], product_name[Dell monitor, Dell keyboard, iPhone6s], product_price[795.62, 104, 560.93], product_color[Red, Yellow, Red], product_size[Medium, Medium, Small], product_description[Latest model!, Keyboard for games!, Second hand!])
# # Customers(customer_id[1, 2, 3], gender_code[Female, Male, Male], customer_first_name[Carmen, Jace, Vickie], customer_middle_initial[K, P, B], customer_last_name[Treutel, Mraz, Bergnaum], email_address[pgulgowski@example.com, zwisozk@example.org, herzog.imogene@example.org], login_name[murphy07, desmond.steuber, kihn.alfonso], login_password[58952d0e0d28de32db3b, 7ba2e47aa0904d9fbdbf, 83a1afbe21f5ca4cd2d5], phone_number[(253)336-6277, 628-468-4228x5917, 633-223-0975], address_line_1[646 Herzog Key Suite 702, 67899 Cassin Hollow Suite 071, 395 Christophe Trail], town_city[Port Madonnahaven, Port Korychester, Lornaland], county[Israel, Palau, Moldova], country[USA, USA, USA])
# # Customer_Payment_Methods(customer_id[15, 1, 10], payment_method_code[Direct Debit, Direct Debit, Direct Debit])
# # Invoices(invoice_number[1, 2, 3], invoice_status_code[Paid, Issued, Paid], invoice_date[2018-03-09 07:16:07, 2018-01-28 20:08:22, 2018-02-13 02:16:55])
# # Orders(order_id[1, 2, 3], customer_id[5, 13, 13], order_status_code[Cancelled, Part Completed, Cancelled], date_order_placed[2017-09-17 16:13:07, 2017-10-14 12:05:48, 2017-09-10 08:27:04])
# # Order_Items(order_item_id[1, 2, 3], product_id[4, 3, 2], order_id[8, 4, 7], order_item_status_code[Delivered, Out of Stock, Delivered])
# # Shipments(shipment_id[1, 2, 3], order_id[7, 6, 9], invoice_number[5, 2, 4], shipment_tracking_number[6900, 3499, 5617], shipment_date[2018-02-28 00:04:11, 2018-03-07 01:57:14, 2018-03-18 22:23:19])
# # Shipment_Items(shipment_id[4, 7, 15], order_item_id[4, 14, 9])
# #
# ### Foreign key information of SQLite tables, used for table joins:
# #
# # Customer_Payment_Methods(customer_id) REFERENCES Customers(customer_id)
# # Orders(customer_id) REFERENCES Customers(customer_id)
# # Order_Items(order_id) REFERENCES Orders(order_id)
# # Order_Items(product_id) REFERENCES Products(product_id)
# # Shipments(order_id) REFERENCES Orders(order_id)
# # Shipments(invoice_number) REFERENCES Invoices(invoice_number)
# # Shipment_Items(order_item_id) REFERENCES Order_Items(order_item_id)
# # Shipment_Items(shipment_id) REFERENCES Shipments(shipment_id)
# #
# ### Some example pairs of questions and corresponding SQL queries are provided based on similar questions:
# ### How many farms are there?
# SELECT count(*) FROM farm
# ### What is the average, minimum, and maximum age for all French singers?
# SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
# ### Show the ID of the high schooler named Kyle.
# SELECT ID FROM Highschooler WHERE name  =  'Kyle'
# #
# ### What are the names and colors of all products that have been shipped?
# """

# messages = [
#     {"role": "user", "content": prompt0},
#     {"role": "user", "content": prompt1},
# ]

In [21]:
model = "mlx-community/Llama-3.2-3B-Instruct-4bit"
output_file = f"{PRED_PATH}/pred.sql"

In [23]:
MAX_TOKENS = 512

print(f"Starting batch inference with {len(prompts)} prompts")
print("=" * 60)

model, tokenizer = load_model(model)

results = process_batch(
    prompts=prompts,
    model=model,
    tokenizer=tokenizer,
    max_tokens=MAX_TOKENS
)

save_results(results, output_file)

successful = sum(1 for r in results if r['status'] == 'success')
failed = len(results) - successful
total_time = sum(r['generation_time'] for r in results)

print("=" * 60)
print("BATCH PROCESSING COMPLETE")
print(f"Total prompts: {len(prompts)}")
print(f"Successful: {successful}")
print(f"Failed: {failed}")
print(f"Total time: {total_time:.2f}s")
print(f"Average time per prompt: {total_time/len(prompts):.2f}s")
print(f"Results saved to {output_file}")

Starting batch inference with 1 prompts
Loading model: mlx-community/Llama-3.2-3B-Instruct-4bit


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 60061.63it/s]


Model loaded successfully!
Processing prompt 1/1
Completed in 3.12s
--------------------------------------------------
Results saved to /Users/atissera/Developer/repos/unrc-cs-thesis/data/predictions/pred.sql
BATCH PROCESSING COMPLETE
Total prompts: 1
Successful: 1
Failed: 0
Total time: 3.12s
Average time per prompt: 3.12s
Results saved to /Users/atissera/Developer/repos/unrc-cs-thesis/data/predictions/pred.sql


In [16]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/Llama-3.2-3B-Instruct-4bit")

prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)

generate(model, tokenizer, prompt=prompt, verbose=True)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 38187.90it/s]


SELECT T1.product_name, T1.product_color FROM Products AS T1 INNER JOIN Shipments AS T2 ON T1.product_id = T2.order_item_id
Prompt: 2511 tokens, 556.967 tokens-per-sec
Generation: 34 tokens, 46.683 tokens-per-sec
Peak memory: 18.326 GB


'SELECT T1.product_name, T1.product_color FROM Products AS T1 INNER JOIN Shipments AS T2 ON T1.product_id = T2.order_item_id'